In [1]:
from __future__ import division, print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
from matplotlib.backends.backend_pdf import PdfPages
from functools import partial, reduce
from itertools import combinations
from scipy.stats import percentileofscore
import affinity as af
import ratlib as rat

%matplotlib inline 

In [2]:
data = pd.read_excel('~/GIT/collinearity/database/sidis/expdata/1008.xlsx')
tab = pd.read_excel('~/GIT/collinearity/notebooks/SIDIS_Proximity/1008_Affinity.xlsx')

In [3]:
class Rfilter(object):

    def __init__(self, hadron='pi+', fudge=[0, 0]):
        #self.M = 0.938
        self.M = 0.94
        self.M2 = self.M**2
        self.set_Mh(hadron)
        

        self.fudge1 = fudge[0]
        self.fudge2 = fudge[1]

        self.MiT2 = 0.5 + 0.3 * fudge[0]
        self.MfT2 = 0.5 + 0.3 * fudge[1]
        self.kT = 0
    
        self.MX = 1.3
        self.Ma = 1.5
        self.Mb = 0.3
        self.deltaM = 0.3
        self.MJ = 0.3
        
        self.MiT = self.MiT2**0.5
        self.MfT = self.MfT2**0.5
        self.kT2 = self.kT**2
        
        # from paper, we need kf2, deltakT2, ki2
        # we need to have a way to vary them!
        self.kf       = 0
        self.ki       = 0
        self.kf2      = self.kf**2
        self.ki2      = self.ki**2
        self.deltakT  = 0
        self.deltakT2 = self.deltakT**2
        
    def set_ki(self,ki):
        self.ki = ki
        self.ki2 = ki**2
        
    def set_kf(self,kf):
        self.kf = kf
        self.kf2 = kf**2
        
    def set_kT(self,kT):
        self.kT = kT
        self.kT2 = kT**2
        
    def set_deltakT(self,deltakT):
        self.deltakT = deltakT
        self.deltakT2 = self.deltakT**2
        
    def set_Mh(self, hadron):
        if hadron == 'pi+':
            self.Mh = 0.135  #  
        if hadron == 'pi-':
            self.Mh = 0.135  #  
        if hadron == 'pi0':
            self.Mh = 0.135  #  
        if hadron == 'k+':
            self.Mh = 0.493
        if hadron == 'k-':
            self.Mh = 0.493
        if hadron == 'k+':
            self.Mh = 0.493
        if hadron == 'h+':
            #self.Mh = 0.135
            self.Mh = 0.14
        if hadron == 'h-':
            self.Mh = 0.135
        self.Mh2 = self.Mh**2
   
    def set_kibT(self,kibT):
        self.kibT = kibT
        self.kibT2 = kibT**2
        
    def set_phi(self,phi):
        self.phi = phi
    
    def set_xi(self,xi):
        self.xi = xi
        
    def set_zeta(self,zeta):
        self.zeta = zeta
    
    def set_MBT(self,PhT):
        self.MBT = np.sqrt( self.Mh2 + (PhT**2 ))
        
    def set_xn(self, x, Q2):
        self.xn = (2 * x / (1 + np.sqrt(1 + 4 * x**2 * self.M2 / Q2)))
    
    def set_zn(self, x, z, Q2, PhT, hadron):
        self.set_xn(x,Q2)
        self.set_Mh(hadron)
        self.set_MBT(PhT)
        self.zn =  self.xn * z/ (2 * x) * ( 1. + np.sqrt( 1.- 4. * self.M2 * self.MBT**2 * x**2 /  ( Q2**2 * z**2) ) )
        
    def set_MfT(self):
        self.MfT = np.sqrt(self.kT**2 + self.MJ**2 +
                           self.deltaM**2 * self.fudge2)
    def set_MiT(self,x,Q2):
        self.set_xn(x, Q2)
        self.MiT = np.sqrt((self.xn * self.kT**2 + self.xn * (self.Ma + self.Mb / np.sqrt(self.xn))
                            ** 2 - (1 - self.xn) * self.xn * self.M2 + self.deltaM**2 * self.fudge1) / (1 - self.xn))
    def set_W2(self,x,Q2):
        self.W2 = Q2 * (1. - x) / x + self.M2
    
    def set_yh(self, x, z, Q2, PhT, hadron, sign=-1):
        self.set_xn(x, Q2)
        self.set_Mh(hadron)
        expy = Q2**0.5 * z * (Q2 - self.xn**2 * self.M2)\
            / (2 * self.M2 * self.xn**2 * (self.Mh2 + PhT**2)**0.5)\
            + sign * Q2**0.5 / (self.xn * self.M) * (z**2 * (Q2 - self.xn**2 * self.M2)**2
                                        / (4 * self.M2 * self.xn**2 * (self.Mh2 + PhT**2)) - 1)**0.5
        self.yh = np.log(expy)
        
    def get_ki2(self):
        return self.ki2
    
    def get_kf(self):
        return self.kf
    
    def get_W2(self):
        return self.W2

    def get_MBT(self):
        return self.MBT
        
    def get_MiT(self):
        return self.MiT

    def get_MfT(self):
        return self.MfT

    def get_xn(self):
        return self.x

    def get_yh(self):
        return self.yh

    def get_zn(self):
        return self.zn
        
# rapidity of the target

    def get_yp(self, x, Q2):
        self.set_xn(x, Q2)
        return np.log(np.sqrt(Q2) / (self.xn * self.M))

    def get_yi(self, Q2):
        return 0.5 * np.log(Q2 / self.MiT**2)

    def get_yf(self, Q2):
        return -0.5 * np.log(Q2 / self.MfT**2)

    def get_MhT(self, PhT):
        return np.sqrt(self.Mh2 + PhT**2)
    
    def get_kibT(self,kibT):
        return self.kibT
    
    def get_M(self):
        return self.M
    
    def get_Mh(self):
        return self.Mh
    
    def get_Mh2(self):
        return self.Mh2
    
    def get_R(self, x, z, Q2, PhT, hadron):
        
        self.set_zn(x, z, Q2, PhT, hadron)
        zn2 = self.zn**2
        self.set_xn(x, Q2)
        znhat = self.zn/self.zeta
        xnhat = self.xn/self.xi
        qT = -PhT/z
        
        # from paper..
        Ph_kf = ((((self.Mh2+zn2*qT**2)*znhat)/(2*self.zn))+(self.zn*((-znhat*qT+self.deltakT)**2+ \
                    self.kf2)/(2*znhat))) + self.zn*qT*(-znhat*qT+self.deltakT)
        Ph_ki = ((((self.Mh2+zn2*qT**2)*(xnhat*(self.ki2 + self.kibT2)))/(2*Q2*self.zn)) + \
                 (self.zn*Q2)/(2*xnhat))+self.zn*qT*self.kibT
        
        return np.abs(Ph_kf / Ph_ki)
    
    def get_R1S(self, x, z, Q2, PhT, hadron):
        
        self.set_zn(x, z, Q2, PhT, hadron)
        zn2 = self.zn**2
        self.set_xn(x, Q2)
        znhat = self.zn/self.zeta
        xnhat = self.xn/self.xi
        qT = -PhT/z
        
        R1 = 1/self.zeta*(self.Mh2+self.zeta**2*(self.deltakT2+self.kf2))/((self.Mh2+zn2*qT**2)* \
             (self.ki2+self.kibT**2)*(self.xn/(self.zn*Q2*self.xi))+ (self.zn*Q2*self.xi/self.xn) + \
             2*self.zn*qT*self.kibT*np.cos(self.phi))
        return R1
        
        
# R0, Eq. (4.14)...
    def get_R0(self, Q2):
        """
        Collinearity ratio defined in the paper Eq. (4.15)
        """
        R0 = (np.maximum(np.maximum(self.ki**2/Q2,self.kf**2/Q2),self.kT**2/Q2))
        #print(R0)
        return  R0
        
        
# We call R1 in the new paper what was R in the previous...
    def get_R1(self, x, z, Q2, PhT, hadron):
        Rval.set_Mh(hadron)
        """
        Collinearity ratio defined in the paper Eq. (4.15)
        """
        return  self.get_R(x, z, Q2, PhT, hadron)

# R2 from Eq. (4.17)        
    def get_R2(self, x, z, Q2, PhT, hadron):
        self.set_Mh(hadron)
        self.set_zn(x, z, Q2, PhT, hadron)
        znhat = self.zn/self.zeta
        qT = -PhT/z
        # from paper..
        return(np.abs( -(1.-znhat) - znhat*qT**2/Q2 - (1.-znhat)*self.kf2/(znhat*Q2) - \
                      self.deltakT2/(znhat*Q2) + 2.*qT*self.deltakT/Q2 ))
    
   


In [4]:
%%time

hadron = 'h+'
fudge = [0,0]
Rval = Rfilter(hadron = hadron,fudge = fudge)
Rval.set_kT(.1)
Rval.set_ki(.1)
Rval.set_kf(.1)
Rval.set_deltakT(.3)
Rval.set_kibT(0.2)
Rval.set_xi(0.3)
Rval.set_zeta(0.3)
Rval.set_phi(0)

x = .2
Q2 = 3**2
z = .3
pT = .1
M = 0.94
M_h = .14

R1 = Rval.get_R1(x, z, Q2, pT, 'h+')
R2 = Rval.get_R2(x, z, Q2, pT, 'h+')
R1S2 = Rval.get_R1S( x, z, Q2, pT, hadron)
print('Rfilter     ',R1,'',R2)
print('Rfilter R1S ',R1S2)

# From ratios.py
M= .94
M_h=.14
x_bj = .2
Q = 3
z_h = .3
P_t = 0.1

    
xi = .3
zeta = .3
delta_k_t = .3 
k_i_t = .2 
M_ki = .1 
M_kf = .1 
q_t = -P_t/z_h
eta = .1

R1S = rat.get_R1( M,M_h,x_bj,z_h,eta,Q,q_t,xi,zeta,delta_k_t,k_i_t,M_ki,M_kf)
R2S = rat.get_R2(M,M_h,x_bj,z_h,eta,Q,q_t,xi,zeta,delta_k_t,k_i_t,M_ki,M_kf)

print('ratios.py   ',R1S,'',R2S)

Rfilter      0.023774226003570147  0.048602633009976616
Rfilter R1S  0.02377422600357015
ratios.py    0.023775081134120253  0.04860224399921939
CPU times: user 488 µs, sys: 28 µs, total: 516 µs
Wall time: 428 µs


In [5]:
# Proximity is defined for a sphere of a selected radius(p) by the following function
# This notebook does not use this function to compare ratios.py with Rfilter

def Proximity(data,domain1,domain2,domainPhi,p,N,fudge):
    """
    data    = any Pandas DataFrame should do as long as it contains x,z,Q2, and pT
    domain1 = 
    domain2 = 
    p       = radius of sphere representing the demarcation of the proximity
    hadron  = any hadron found in the data
    fudge   = idk what fudge is so I did the right thing and passed it along 
    """
    
    #data.loc[:,'proximity'] = 0.
    
    
    for i in range(len(data)):
        
        hadron = data.loc[i,'hadron']
        Rval = Rfilter(hadron = hadron,fudge = fudge)
        # Creates vectors of length N made from uniform random numbers in defined domains.
        zeta = np.random.uniform(data.loc[i,'z']-.1,data.loc[i,'z']+.1,N)
        ki = np.random.uniform(domain1[0],domain1[1],N)
        kf = np.random.uniform(domain1[0],domain1[1],N)
        kT = np.random.uniform(domain1[0],domain1[1],N)
        kibT = np.random.uniform(domain1[0],domain1[1],N)
        deltakT = np.random.uniform(domain2[0],domain2[1],N)
        xi = np.linspace(domain2[0],domain2[1],N)
        phi = np.linspace(domainPhi[0],domainPhi[1],N)
        
        # Sets our parameters for Rfilter
        Rval.set_ki(ki)
        Rval.set_kf(kf)  
        Rval.set_kT(kT)
        Rval.set_kibT(kibT)
        Rval.set_deltakT(deltakT)
        Rval.set_zeta(zeta)
        Rval.set_phi(phi)
        Rval.set_xi(xi)
        
        # Gets Ratios from Rfilter 
        R0 = Rval.get_R0(data.loc[i,'Q2'])             
        R1 = Rval.get_R1(data.loc[i,'x'], data.loc[i,'z'], data.loc[i,'Q2'], data.loc[i,'pT'], hadron)
        R2 = Rval.get_R2(data.loc[i,'x'], data.loc[i,'z'], data.loc[i,'Q2'], data.loc[i,'pT'], hadron)
        SR1 = Rval.get_R1S(data.loc[i,'x'], data.loc[i,'z'], data.loc[i,'Q2'], data.loc[i,'pT'], hadron)
        
        # proximity uses Rfiler for R0, R1, R2 with Alexei's method of proximity using error to define what
        # is inside sphere of radius p(typically p = 1)
        data.loc[i,'proximity'] = sum(1 for i in range(N) if (R0[i]**2+R1[i]**2+R2[i]**2)<p)/N
        
        # affinity uses Rfilter for R1 and R2 but with Sterlings approach for Saffinity %
        data.loc[i,'affinity']  = sum([1 for i in range(N) if SR1[i]<0.3 if R2[i]<0.3])/N  
        
        # Input for Sterlings code
        x=data.loc[i,'x']
        z=data.loc[i,'z']
        Q=data.loc[i,'Q2']**.5
        val=data.loc[i,'value']
        P_t=data.loc[i,'pT']

        q_t = -P_t/z
        M= Rval.get_M()
        M_h = Rval.get_Mh()
        #M_h=.14
        eta = rat.get_eta( M,M_h,x,z,Q,q_t)
        
        sR1 = rat.get_R1( M,M_h,x_bj,z_h,eta,Q,q_t,xi,zeta,deltakT,kibT,ki,kf)
        sR2 = rat.get_R2( M,M_h,x_bj,z_h,eta,Q,q_t,xi,zeta,deltakT,kibT,ki,kf)
        
        # Saffinity uses Sterlings equations in ratlib.py
        data.loc[i,'Saffinity'] = sum([1 for i in range(N) if sR1[i]<0.3 if sR2[i]<0.3])/N 
        if i == 2:
            for j in range(len(R1)):
                print('R1 Rfilter =          ',R1[j],'\nR1 Sterling Rfilter = ',SR1[j],'\nratlib R1 =           ',sR1[j],'\n')
    return data

In [6]:
%%time
N = 1000
p = 1
fudge = [0,0]
domain1 = [0,.1]
domain2 = [0.3,0.4]
domainPhi = [0,0]
data = Proximity(data,domain1,domain2,domainPhi,p,N,fudge)

R1 Rfilter =           0.021638495210182976 
R1 Sterling Rfilter =  0.021638495210182997 
ratlib R1 =            0.018384030002562 

R1 Rfilter =           0.01967920383848722 
R1 Sterling Rfilter =  0.019679203838487234 
ratlib R1 =            0.016733000993919014 

R1 Rfilter =           0.013790028668472948 
R1 Sterling Rfilter =  0.013790028668472953 
ratlib R1 =            0.011740353990380712 

R1 Rfilter =           0.013996055395243588 
R1 Sterling Rfilter =  0.013996055395243562 
ratlib R1 =            0.011887517974933478 

R1 Rfilter =           0.017586881316604366 
R1 Sterling Rfilter =  0.017586881316604394 
ratlib R1 =            0.014958481828362123 

R1 Rfilter =           0.020845756367427325 
R1 Sterling Rfilter =  0.020845756367427318 
ratlib R1 =            0.017723271216327108 

R1 Rfilter =           0.02478212020139912 
R1 Sterling Rfilter =  0.024782120201399173 
ratlib R1 =            0.02107066093029199 

R1 Rfilter =           0.02005695682045399 
R1 Sterlin

R1 Rfilter =           0.012519272793185038 
R1 Sterling Rfilter =  0.012519272793185023 
ratlib R1 =            0.010645318102849778 

R1 Rfilter =           0.018911175599535816 
R1 Sterling Rfilter =  0.018911175599535836 
ratlib R1 =            0.01606333313970347 

R1 Rfilter =           0.022934777030232036 
R1 Sterling Rfilter =  0.02293477703023203 
ratlib R1 =            0.019515983274239885 

R1 Rfilter =           0.021122776917049762 
R1 Sterling Rfilter =  0.021122776917049724 
ratlib R1 =            0.0179638419840671 

R1 Rfilter =           0.014987699863713242 
R1 Sterling Rfilter =  0.014987699863713264 
ratlib R1 =            0.012756164520247833 

R1 Rfilter =           0.01321625036312492 
R1 Sterling Rfilter =  0.013216250363124911 
ratlib R1 =            0.011244271539454261 

R1 Rfilter =           0.014742367195662775 
R1 Sterling Rfilter =  0.014742367195662784 
ratlib R1 =            0.01252198754367906 

R1 Rfilter =           0.012378767709055685 
R1 Sterli

R1 Sterling Rfilter =  0.010452049446127322 
ratlib R1 =            0.008890299825451479 

R1 Rfilter =           0.012477960439626302 
R1 Sterling Rfilter =  0.012477960439626276 
ratlib R1 =            0.01061883298654544 

R1 Rfilter =           0.015180603879272581 
R1 Sterling Rfilter =  0.01518060387927256 
ratlib R1 =            0.012906028335664005 

R1 Rfilter =           0.011945862080241879 
R1 Sterling Rfilter =  0.011945862080241879 
ratlib R1 =            0.010150246157865209 

R1 Rfilter =           0.020366805838653485 
R1 Sterling Rfilter =  0.020366805838653503 
ratlib R1 =            0.017315241697948753 

R1 Rfilter =           0.017799111148453325 
R1 Sterling Rfilter =  0.017799111148453336 
ratlib R1 =            0.015133127521892458 

R1 Rfilter =           0.014484815719459044 
R1 Sterling Rfilter =  0.014484815719459056 
ratlib R1 =            0.012303659371076574 

R1 Rfilter =           0.011794225090500644 
R1 Sterling Rfilter =  0.011794225090500661 
ratli

R1 Sterling Rfilter =  0.015672593508290925 
ratlib R1 =            0.013327293064594101 

R1 Rfilter =           0.012094142212214627 
R1 Sterling Rfilter =  0.012094142212214613 
ratlib R1 =            0.01028144829748159 

R1 Rfilter =           0.013790976817203354 
R1 Sterling Rfilter =  0.013790976817203348 
ratlib R1 =            0.011724665843324008 

R1 Rfilter =           0.011108869470452826 
R1 Sterling Rfilter =  0.011108869470452837 
ratlib R1 =            0.00944385896599344 

R1 Rfilter =           0.011854911446389947 
R1 Sterling Rfilter =  0.011854911446389943 
ratlib R1 =            0.01007888656083317 

R1 Rfilter =           0.012880391665063731 
R1 Sterling Rfilter =  0.012880391665063724 
ratlib R1 =            0.010949477793150064 

R1 Rfilter =           0.011218808228399805 
R1 Sterling Rfilter =  0.011218808228399818 
ratlib R1 =            0.009533190157166813 

R1 Rfilter =           0.019183649389251448 
R1 Sterling Rfilter =  0.01918364938925144 
ratlib 

CPU times: user 9.88 s, sys: 56.6 ms, total: 9.93 s
Wall time: 9.83 s


In [7]:
#data.to_excel('proximityTest_Nov12.xlsx')

In [8]:
data

,Ebeam,x,Q2,y,z,pT,pT2,obs,value,stat_u,sys_u,target,hadron,col,F2,proximity,affinity,Saffinity
0,160,0.1570,20.0,0.439,0.2,0.300000,0.09,M_Compass,6.2719,0.3126,0.1051,deuteron,h+,compass,0.64392,1.0,0.499,0.511
1,160,0.1570,20.0,0.439,0.2,0.331662,0.11,M_Compass,6.2175,0.3135,0.1047,deuteron,h+,compass,0.64392,1.0,0.490,0.551
2,160,0.1570,20.0,0.439,0.2,0.360555,0.13,M_Compass,5.0537,0.2711,0.1070,deuteron,h+,compass,0.64392,1.0,0.455,0.595
3,160,0.1570,20.0,0.439,0.2,0.400000,0.16,M_Compass,4.8854,0.2244,0.1067,deuteron,h+,compass,0.64392,1.0,0.445,0.618
4,160,0.1570,20.0,0.439,0.2,0.424264,0.18,M_Compass,4.1757,0.2275,0.1101,deuteron,h+,compass,0.64392,1.0,0.453,0.629
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2327,160,0.0389,1.4,0.122,0.6,1.330413,1.77,M_Compass,0.0057,0.0009,0.1207,deuteron,h+,compass,0.00000,0.0,0.000,0.000
2328,160,0.0389,1.4,0.122,0.6,1.396424,1.95,M_Compass,0.0033,0.0006,0.1367,deuteron,h+,compass,0.00000,0.0,0.000,0.000
2329,160,0.0389,1.4,0.122,0.6,1.483240,2.20,M_Compass,0.0014,0.0003,0.1765,deuteron,h+,compass,0.00000,0.0,0.000,0.000
2330,160,0.0389,1.4,0.122,0.6,1.581139,2.50,M_Compass,0.0016,0.0004,0.1795,deuteron,h+,compass,0.00000,0.0,0.000,0.000
